# Making Python respect Docker memory limits

* [Making Python respect Docker memory limits](https://carlosbecker.com/posts/python-docker-limits/)

> If you run Python inside containers, chances are you have seen Linux’s OOMKiller working at least a couple of times. This happens because Python sees the entire host’s resources as if they were available for its use. Then, it may try to allocate more memory than it is allowed to, which causes Linux to kill the process.
> 
> To fix that, we may read the actual limits from /sys/fs/cgroup/memory/memory.limit_in_bytes and set it as the process max address space area.

## Example:

```
[55850.330482] oom-kill:constraint=CONSTRAINT_NONE,nodemask=(null),cpuset=docker-dca91626d286fd03991cb36a560bf922b7280fe745353d32d3c78fab5e54d6e2.scope,mems_allowed=0,global_oom,task_memcg=/system.slice/docker-dca91626d286fd03991cb36a560bf922b7280fe745353d32d3c78fab5e54d6e2.scope,task=python3,pid=122850,uid=0
[55850.330551] Out of memory: Killed process 122850 (python3) total-vm:90356148kB, anon-rss:27890556kB, file-rss:0kB, shmem-rss:0kB, UID:0 pgtables:98344kB oom_score_adj:0
```


# For cgroup v1

cgroup v2 removed ```/sys/fs/cgroup/memory/memory.limit_in_bytes```.

* [/sys/fs/cgroup/memory/memory.limit_in_bytes: No such file or directory](https://github.com/oracle/docker-images/issues/1939)

>  host (and docker) is configured to use cgroup V2 which is not compatible with cgroup V1

Pervious docker versions relies on cgruop v1:

* [What does docker mean when it says "Memory limited without swap"](https://stackoverflow.com/a/63726105/4281353)

> Docker daemon relies on the following virtual files to implement memory and swap limits:
> ```
> /sys/fs/cgroup/memory/memory.limit_in_bytes
> /sys/fs/cgroup/memory/memory.memsw.limit_in_bytes
> ```
> If your kernel does not support swap memory limit, the second file won't be there, and docker run won't impose any limitations on the use of the swap space. That way the container is even allowed to use more swap than the -m, --memory setting, as if --memory-swap had been set to -1. Obviously, the container can't use more swap space than you have configured on your system.
> 
> Ubuntu/Debian containers at ```/sys/fs/cgroup/memory/memory.stat```

In [ ]:
import resource
import os

if os.path.isfile('/sys/fs/cgroup/memory/memory.limit_in_bytes'):
    with open('/sys/fs/cgroup/memory/memory.limit_in_bytes') as limit:
        mem = int(limit.read())
        resource.setrlimit(resource.RLIMIT_AS, (mem, mem))


# Generic

* [Dask system.py](https://github.com/shughes-uk/distributed/blob/master/distributed/system.py)

In [ ]:
import sys

import psutil

__all__ = ("memory_limit", "MEMORY_LIMIT")


def memory_limit() -> int:
    """Get the memory limit (in bytes) for this system.

    Takes the minimum value from the following locations:

    - Total system host memory
    - Cgroups limit (if set)
    - RSS rlimit (if set)
    """
    limit = psutil.virtual_memory().total

    # Check cgroups if available
    # Note: can't use LINUX and WINDOWS constants as they upset mypy
    if sys.platform == "linux":
        try:
            # cgroups v1 hard limit
            with open("/sys/fs/cgroup/memory/memory.limit_in_bytes") as f:
                cgroups_limit = int(f.read())
            if cgroups_limit > 0:
                limit = min(limit, cgroups_limit)
        except Exception:
            pass
        try:
            # cgroups v1 soft limit
            with open("/sys/fs/cgroup/memory/memory.soft_limit_in_bytes") as f:
                cgroups_limit = int(f.read())
            if cgroups_limit > 0:
                limit = min(limit, cgroups_limit)
        except Exception:
            pass
        try:
            # cgroups v2 hard limit
            with open("/sys/fs/cgroup/memory.max") as f:
                cgroups_limit = int(f.read())
                if cgroups_limit > 0:
                    limit = min(limit, cgroups_limit)
        except Exception:
            pass
        try:
            # cgroups v2 soft limit
            with open("/sys/fs/cgroup/memory.high") as f:
                cgroups_limit = int(f.read())
                if cgroups_limit > 0:
                    limit = min(limit, cgroups_limit)
        except Exception:
            pass

    # Check rlimit if available
    if sys.platform != "win32":
        try:
            import resource

            hard_limit = resource.getrlimit(resource.RLIMIT_RSS)[1]
            if hard_limit > 0:
                limit = min(limit, hard_limit)
        except (ImportError, OSError):
            pass

    return limit


MEMORY_LIMIT = memory_limit()